<a href="https://colab.research.google.com/github/CarloRomeoGitHub/ML-exercises/blob/master/ProgettoDataAnalyticsCarloRomeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questo progetto verte sul problema della Multiclassificazione. Fa riferimento al dataset [*Intel-Image-Classification*](https://www.kaggle.com/puneet6060/intel-image-classification) di Kaggle.
Questo dataset e' composto da sei classi riguardanti diversi paesaggi. Abbiamo:


1.   Costruzioni
2.   Mare
3.   Montagna
4.   Ghiacciai
5.   Urbano
6.   Foresta

Le immagini che compongono il dataset sono delle immagini a colori di diversa misura che sono state opportunamente ridimensionate prima della fase di training.
Per errori presenti nella porzione *Previsioni*, ho deciso di utilizzare la porzione *Test* per validazione e testing dividendola in parti uguali.
L'obiettivo principale di questo progetto non e' la pretesa di sviluppare un motore ad elevate prestazioni ma, attraverso una corposa fase di tuning, capire come la rete si comporta con diverse configurazioni.




Codice necessario per l'implementazione di TensorBoard.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

--2020-02-09 12:53:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 50.17.165.171, 34.206.126.139, 34.196.175.208, ...
Connecting to bin.equinox.io (bin.equinox.io)|50.17.165.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.2MB/s    in 0.7s    

2020-02-09 12:54:00 (18.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
import os
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

TensorFlow 2.x selected.


Using TensorFlow backend.


Utilizziamo questa funzione per pulire direttamente il Drive sul quale stiamo lavorando.

In [0]:
import shutil

shutil.rmtree('../content/drive/My Drive/fold')

Tramite ZipFile possiamo estrarre lo zip precedentemente caricato sul Drive in una specifica cartella chiamata *fold* nel nostro caso.


In [0]:
with ZipFile('../content/drive/My Drive/intel-image-classification.zip', 'r') as zipObj:
  zipObj.extractall('../content/drive/My Drive/fold')

Con questa funzione, tramite il parametro *directory* assegnamo le labels estrapolandole direttamente dal nome della cartella.

In [0]:
def get_images(directory):
    Images = []
    Labels = []  
    label = 0
    
    for labels in os.listdir(directory): #Tramite il parametro directory definiamo ogni cartella con il nome della label
        if labels == 'buildings': 
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'glacier':
            label = 2
        elif labels == 'mountain':
            label = 3
        elif labels == 'sea':
            label = 4
        elif labels == 'street':
            label = 5
        
        for image_file in os.listdir(directory+labels): #Componiamo il nome dell'immagine partendo dal Folder e dalla Label
            image = cv2.imread(directory+labels+r'/'+image_file) 
            image = cv2.resize(image,(150, 150))
            image = image / 255 #Normalizzazione effettuata solo per il Modello 10   
            Images.append(image)
            Labels.append(label)
    
    return shuffle(Images,Labels,random_state=42) 

def get_classlabel(class_code):
    labels = { 0:'buildings', 1:'forest', 2:'glacier', 3:'mountain',  4:'sea', 5:'street' }
    
    return labels[class_code]

Abbiamo estrapolato separatamente le immagini dai due insiemi in modo da non sovraccaricare la RAM, evitando cosi' eventuali perdite di dati.

In [0]:
#Estrazione Dati
Images_train, Labels_train = get_images('../content/drive/My Drive/fold/seg_train/seg_train/')

Images_train = np.array(Images_train)
Labels_train = np.array(Labels_train)



In [0]:
Images_test, Labels_test = get_images('../content/drive/My Drive/fold/seg_test/seg_test/')

numb = int(len(Images_test)/2)

Images_val = Images_test[:numb]
Images_val = np.array(Images_val)

Labels_val = Labels_test[:numb]
Labels_val = np.array(Labels_val)

Images_test = Images_test[numb:]
Images_test = np.array(Images_test)

Labels_test = Labels_test[numb:]
Labels_test = np.array(Labels_test)


Prima di lavorare sui dati visualizziamo qualche esempio prendendo qualche immagine random  dal nostro dataset.

In [0]:
#Visualizzazione dei Dati
fig = plot.figure(figsize=(30, 30))
f,ax = plot.subplots(5,5) 
f.subplots_adjust(0,0,3,3)
for i in range(0,5,1):
    for j in range(0,5,1):
        rnd_number = randint(0,len(Images_train))
        ax[i,j].imshow(Images_train[rnd_number])
        ax[i,j].set_title(get_classlabel(Labels_train[rnd_number]))
        ax[i,j].axis('off')


Abbiamo utilizzato tre diverse callback per poter sfruttare al meglio il motore.

1.   *EarlyStopper* viene utilizzata per evitare l'overfitting, questa fermera' il motore se dopo x epoche ( x viene definito dal parametro *patience*) la loss non decrementa. E' stato utilizzato inoltre il parametro *restore_best_weights* in modo da poter ripristinare i pesi migliori;

2.  *TensorBoard* viene utilizzata per accedere all'omonimo tool, il quale fornisce una visualizzazione approfondita della rete;

3.   *ModelCheckPoint* permette di salvare checkpoint del modello al termine di ogni epoca. Questo previene la perdita di dati per esempio per una disconnessione. Questa callback viene definita all'interno di ogni motore e non in maniera generica (come fatto per le altre) in modo da poter salvare ogni modello con un differente nome in modo immediato.





In [0]:
#CallBack

earlystopper = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)

tensorboard_callback = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=100,
                         write_images=True)

Utilizziamo il metodo *load_model* in modo da poter caricare un modello precedentemente allenato per poterlo visualizzare tramite TensorBoard.

In [0]:
model = models.load_model('/content/model10Log45.h5')

Questa codice permette di accedere alla propria sessione di TensorBoard.

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://bd9281e8.ngrok.io


Qui abbiamo definito una *GridSearch* per poter trovare gli iperparamentri della rete, scandendo diverse combinazioni.
Gli aspetti presi in considerazione sono:


1.   Numero di filtri per ogni layer convoluzionale
2.   Dimensione di ogni filtro
3.   Presenza dello strides
4.   Funzione di attivazione
5.   Unita' di ogni layer denso
6.   Valore del drop out

Il modello con Grid Search per carenza di risorse computazionali, sebbene sia stato effettuato un tuning dei parametri esaminabili per trovare la configurazione minima, non e' eseguibile in quanto produce un *Memory Leak*.
Per completezza e' stato comunque lasciato il modello.



In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

#GRID SEARCH MODEL
def create_model(first_filt, sec_filt, third_filt, first_kern, sec_kern, third_kern, strides, activation, units, drop_out):
  #DEFAULT PARAMS
  first_filt = 64
  sec_filt = 128
  third_filt = 256
  first_kern = 7
  sec_kern = 5
  third_kern = 3
  strides = 2
  activation = 'elu'
  units = 64
  drop_out = 0.5

  model = Sequential()

  model.add(Conv2D(first_filt, first_kern, activation = activation, padding = "same", input_shape = [150, 150, 3]))
  model.add(Conv2D(first_filt, first_kern, activation = activation, padding = "same"))
  model.add(AvgPool2D(2))

  model.add(Conv2D(sec_filt,sec_kern, activation = activation, padding ="same"))
  model.add(Conv2D(sec_filt,sec_kern, activation = activation, padding ="same", strides= 2)) 
  model.add(MaxPooling2D(2))

  model.add(Conv2D(third_filt,third_kern, activation = activation, padding ="same"))
  model.add(Conv2D(third_filt,third_kern, activation = activation, padding ="same"))
  model.add(MaxPooling2D(2))

  model.add(Flatten())

  model.add(Dense(units, activation = activation))
  model.add(Dropout(drop_out))

  model.add(Dense(units, activation = activation))
  model.add(Dropout(drop_out))

  model.add(Dense(6, activation = "softmax"))

  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  model.summary()
  return model


In [0]:
#EXECUTE GRID SEARCH

first_filt = [64, 128]
sec_filt = [128, 256]
third_filt = [256, 512]
first_kern = [7, 5, 3]
sec_kern = [5, 3]
third_kern = [3, 1]
strides = [2, 0]
activation = ['elu', 'relu']
units = [64, 128]
drop_out = [0.5, 0.3, 0.1]

param_grid = dict(first_filt = first_filt, sec_filt = sec_filt, third_filt = third_filt,
                  first_kern = first_kern, sec_kern = sec_kern, third_kern = third_kern,
                  strides = strides, activation = activation, units = units,
                  drop_out = drop_out)
model = KerasClassifier(build_fn = create_model, batch_size = 100, epochs = 30, verbose = 1)
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = 1)
grid.fit(Images_train, Labels_train)
grid.best_estimator_

Troviamo *batch_size* relativamente basso poiche' valori elevati porterebbero ad un crash della rete a causa delle risorse limitate.
I motori sono stati allenati su cento epoche di partenza. Ogni motore poi, a causa dell'overfitting, ha raggiunto un valore ottimale nell'intorno delle 30 epoche.

In [0]:
#EXECUTION PARAMS
batch_size = 50
epochs = 3

Nella definizione di ogni modello si possono trovare, come commento, le prestazioni raggiunte dal modello in questione durante l'allenamento.
Tale allenamento e' avvenuto sul dataset (Images_train, Labels_train) effettuando una validazione su (Images_val, Labels_val).

In [0]:
#MODEL 1

model = Sequential()

#Epoch 25/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2173 - accuracy: 0.9291
#Epoch 00025: loss improved from 0.21993 to 0.21744, saving model to unet_membrane.hdf5
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 75s 5ms/sample - loss: 0.2174 - accuracy: 0.9290 - val_loss: 0.6126 - val_accuracy: 0.8227
#Epoch 00025: early stopping
#Tests loss:  0.5204133424758911
#Test accuracy:  0.82766664

model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64, 5, activation = "elu", padding = "same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(Conv2D(128,5, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model1.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model1.h5')


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      9472      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      102464    
_________________________________________________________________
average_pooling2d (AveragePo (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 128)       409728    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 256)       2

In [0]:
#MODEL 2

model = Sequential()

#Epoch 31/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.1563 - accuracy: 0.9529
#Epoch 00031: loss did not improve from 0.14833
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 23s 2ms/sample - loss: 0.1563 - accuracy: 0.9529 - val_loss: 0.6264 - val_accuracy: 0.8530
#Epoch 00031: early stopping
#Tests loss:  0.5005115587711334
#Test accuracy:  0.84066665

model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(AvgPool2D(2))

model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(Conv2D(128,5, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model2.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model2.h5')


In [0]:
#MODEL 3

model = Sequential()

#Epoch 31/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.1828 - accuracy: 0.9470
#Epoch 00031: loss did not improve from 0.15335
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 88s 6ms/sample - loss: 0.1827 - accuracy: 0.9471 - val_loss: 0.7764 - val_accuracy: 0.8137
#Epoch 00031: early stopping
#Tests loss:  0.5321324812571208
#Test accuracy:  0.826

model.add(Conv2D(128, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(AvgPool2D(2))

model.add(Conv2D(256,5, activation = "elu", padding ="same"))
model.add(Conv2D(256,5, activation = "elu", padding ="same", strides= 3)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(512,3, activation = "elu", padding ="same"))
model.add(Conv2D(512,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model3.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model3.h5')



In [0]:
#MODEL 4

model = Sequential()

#Epoch 54/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2194 - accuracy: 0.9315
#Epoch 00054: loss improved from 0.22054 to 0.21993, saving model to unet_membrane.hdf5
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 91s 6ms/sample - loss: 0.2199 - accuracy: 0.9315 - val_loss: 1.1133 - val_accuracy: 0.7920
#Epoch 00054: early stopping
#Tests loss:  0.6018184332052866
#Test accuracy:  0.81633335

model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,3, activation = "elu", padding ="same"))
model.add(Conv2D(128,3, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(512,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model4.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model4.h5')


In [0]:
#MODEL 5

model = Sequential()

#Epoch 30/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2361 - accuracy: 0.9238
#Epoch 00030: loss did not improve from 0.21374
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 40s 3ms/sample - loss: 0.2360 - accuracy: 0.9238 - val_loss: 0.7770 - val_accuracy: 0.8253
#Epoch 00030: early stopping
#Tests loss:  0.4971700879732768
#Test accuracy:  0.8383333

model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64,7, activation = "elu", padding ="same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,3, activation = "elu", padding ="same"))
model.add(Conv2D(128,3, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model5.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model5.h5')


In [0]:
#MODEL 6

model = Sequential()
# Epoch 35/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.1534 - accuracy: 0.9537
#Epoch 00035: loss improved from 0.15810 to 0.15335, saving model to unet_membrane.hdf5
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 65s 5ms/sample - loss: 0.1533 - accuracy: 0.9538 - val_loss: 0.7257 - val_accuracy: 0.8337
#Epoch 00035: early stopping
#Tests loss:  0.5391829943656922
#Test accuracy:  0.844

model.add(Conv2D(64, 5, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64,5, activation = "elu", padding ="same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,3, activation = "elu", padding ="same"))
model.add(Conv2D(128,3, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model6.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model6.h5')


In [0]:
#MODEL 7

model = Sequential()

# Epoch 26/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2663 - accuracy: 0.9120
#Epoch 00026: loss did not improve from 0.21993
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 78s 6ms/sample - loss: 0.2660 - accuracy: 0.9121 - val_loss: 0.5647 - val_accuracy: 0.8480
#Epoch 00026: early stopping
#Tests loss:  0.5447170452276866
#Test accuracy:  0.80333334

model.add(Conv2D(64, 5, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64,5, activation = "elu", padding ="same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(Conv2D(128,5, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,5, activation = "elu", padding ="same"))
model.add(Conv2D(256,5, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model7.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model7.h5')


In [0]:
#MODEL 8

model = Sequential()

#Epoch 24/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.1241 - accuracy: 0.9608
#Epoch 00024: loss improved from 0.12924 to 0.12422, saving model to unet_membrane.hdf5
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 68s 5ms/sample - loss: 0.1242 - accuracy: 0.9607 - val_loss: 0.6579 - val_accuracy: 0.8350
#Epoch 00024: early stopping
#Tests loss:  0.4827573082447052
#Test accuracy:  0.849

model.add(Conv2D(64, 5, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64,5, activation = "elu", padding ="same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,3, activation = "elu", padding ="same"))
model.add(Conv2D(128,3, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(256, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.3))

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.1))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model8.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model8.h5')


In [0]:
#MODEL 9 

model = Sequential()

#Epoch 26/100
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2198 - accuracy: 0.9282
#Epoch 00026: loss did not improve from 0.21374
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 40s 3ms/sample - loss: 0.2194 - accuracy: 0.9284 - val_loss: 0.6504 - val_accuracy: 0.8003
#Epoch 00026: early stopping
#Tests loss:  0.47910083723068236
#Test accuracy:  0.8436667


model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64, 5, activation = "elu", padding = "same"))
model.add(AvgPool2D(2))

model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(Conv2D(128,5, activation = "elu", padding ="same", strides= 2)) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Conv2D(512,1, activation = "elu", padding ="same"))
model.add(Conv2D(512,1, activation = "elu", padding ="same"))
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model9.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model9.h5')


In [0]:
#MODEL 10

model = Sequential()

#Epoch 26/50
#14000/14034 [============================>.] - ETA: 0s - loss: 0.2093 - accuracy: 0.9285
#Epoch 00026: loss improved from 0.23279 to 0.20969, saving model to model10.h5
#Restoring model weights from the end of the best epoch
#14034/14034 [==============================] - 40s 3ms/sample - loss: 0.2097 - accuracy: 0.9284 - val_loss: 0.5325 - val_accuracy: 0.8480
#Epoch 00026: early stopping
#Tests loss:  0.45385681772232056
#Test accuracy:  0.83933336

model.add(Conv2D(64, 7, activation = "elu", padding = "same", input_shape = [150, 150, 3]))
model.add(Conv2D(64, 5, activation = "elu", padding = "same"))
model.add(MaxPooling2D(2))

model.add(Conv2D(128,5, activation = "elu", padding ="same"))
model.add(Conv2D(128,5, activation = "elu", padding ="same", strides= 2))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(2))

model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(Conv2D(256,3, activation = "elu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2))

model.add(Conv2D(512,1, activation = "elu", padding ="same"))
model.add(Conv2D(512,1, activation = "elu", padding ="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2))

model.add(Flatten())

model.add(Dense(128, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "elu"))
model.add(Dropout(0.5))

model.add(Dense(6, activation = "softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = ModelCheckpoint('model10.h5', monitor='loss',verbose=1, save_best_only=True)
model.fit(Images_train, Labels_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(Images_val, Labels_val), callbacks=[model_checkpoint, earlystopper, tensorboard_callback])
score = model.evaluate(Images_test, Labels_test, verbose = 0)
print("Tests loss: ", score[0])
print("Test accuracy: ", score[1])
model.save('model10.h5')


Dal confronto tra le diverse perfomance dei dieci motori realizzati emerge che la miglior configurazione realizzata e' l'ultima.
Per la realizzazione di ogni motore e' stata osservata la realizzazione del precedente, traendo da ogni tentativo spunti per il miglioramento della rete.

In [0]:
result = model.predict_classes(Images_test[:10])

In [0]:
print(result, Labels_test[:10])

[0 0 1 1 3 5 4 3 3 4] [0 0 1 1 3 5 4 3 5 4]
